# Tree with state propagation

In [ ]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model import Model

In [ ]:
scd = False

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [ ]:
class SubModel(Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

In [ ]:
sm = SubModel()
sm.state_change_debug = True

In [ ]:
sm.length = 3

Let us define a more complex model

In [ ]:
class InterimModel(Model):
    sm = bu.Instance(SubModel,())
    
    stiffness = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_stiffness(self):
        return self.sm.length * 10
    
    depends_on = ['sm']
    
    ipw_view = bu.View(
        bu.Item('stiffness', readonly=True)
    )

In [ ]:
im = InterimModel(sm=sm)
im.state_change_debug = True

In [ ]:
im.sm.length = 8

In [ ]:
class TopLevelModelEagerGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
    def _sm_default(self):
        sm = SubModel()
        self.set_sm(sm)
        return sm
    def _sm_changed(self):
        self.set_sm(self.sm)
    def set_sm(self, sm):
        self.im.sm = sm
        self.im2.sm = sm
        
    im = bu.Instance(InterimModel, ())
    im2 = bu.Instance(InterimModel, ())

    time = bu.Float(400, ALG=True)

    depends_on = ['sm', 'im', 'im2']
    tree = ['sm', 'im', 'im2']


In [ ]:
tl = TopLevelModelEagerGraphChange()
tl.state_change_debug = True

In [ ]:
tl.sm = SubModel()
tl.sm, tl.im2.sm, tl.im.sm

In [ ]:
tl.im.parents, tl.im2.parents, tl.sm.parents

In [ ]:
tl.im.sm.length = 10

In [ ]:
tl.im, tl.im2, tl.im.sm.parents

In [ ]:
tl.interact()

In [ ]:
im.sm.length

In [ ]:
class TopLevelModelLazyGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
        
    im = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im(self):
        print('im')
        return InterimModel(sm=self.sm)

    im2 = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im2(self):
        print('im2')
        return InterimModel(sm=self.sm)

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']

In [ ]:
tl = TopLevelModelEagerGraphChange()
tl.sm.parents, tl.depends_on, tl.im.depends_on

In [ ]:
tl.interact()

Add change the component and check if the changes of stiffness in InterimModel get updated

In [ ]:
tl.sm = SubModel()
tl.sm.length = 30
tl.im.stiffness